# Isoscape dataset extraction

Author: Martina Kauzlaric (martina.kauzlaric@unibe.ch)

This notebook is used to extract data from monthly isoscapes (i.e. isotopic landscapes, which are spatially continuous and georeferenced isotope datasets) of oxygen-18 [‰] with a resolution of 500m provided by the FOEN into a table for publication alongisde the used data.
Stable isotope data in precipitation of the isotope observation network in Switzerland (ISOT), together with several influencing variables (e.g., topographical parameters, climate variables) are used in a multi-regression framework, and the residuals are interpolated by the use of ordinary kriging.
For now there are only monthly data for the years 2013 and 2020. The remaining years 2007-2023 will be produced and made available by the FOEN by the end of 2025 (Marc Schürch, FOEN, personal communication, 11.11.2024 ).

## Requirements
**Python:**

* Python=3.13.2
* Jupyter
* os
* numpy=2.2.4
* xarray=2024.11.0
* pandas=2.2.3
* geopandas=1.0.1
* cartopy=0.24.1
* matplotlib=3.10.0
* tqdm=4.67.1

Check the Github repository for an environment_landcover.yml (for conda environments) file [for semplicity we use the same environment and that used for extracting the landcover data]

**Files:**

* camels_ch_chem_catchment_boundaries.shp
* GEO+N_13_ASCII & GEO+N_20_ASCII ASCII files


**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 


## References
* https://www.bafu.admin.ch/bafu/en/home/topics/water/groundwater/groundwater-resources/stable-water-isotopes.html
* https://www.bafu.admin.ch/dam/bafu/de/dokumente/hydrologie/externe-studien-berichte/isoscapes_schweiz_endbericht.pdf.download.pdf/isoscapes_schweiz_endbericht.pdf
## Observations
* Data are only available for catchments inside the national boundaries!

# Import modules

In [ ]:
# Clear all variables
%reset -f
#Import necessary libraries
import os
import glob
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon
from shapely.geometry import box
import tqdm as tqdm
import re
from rasterstats import zonal_stats

# Configurations

In [12]:
# Only editable variables:
# Set (relative) path to your local directory
# PATH = ".."
PATH = "S:\\CAMELS-CH\\CAMELS-chem"

In [13]:
## Set directories
GIS_dir = os.path.join(PATH,"data\\GIS")
# Define shapefile with the catchments
catchments_shp = os.path.join(GIS_dir,"shapefile_catchments\\camels_ch_chem_catchment_boundaries.shp")
#Add subfolder to GIS_dir for Isoscape data
GIS_dir = os.path.join(GIS_dir, "Isoscapes")  
PATH_OUTPUT = os.path.join(PATH,"results\\catchment_aggregated_data\\isoscapes")

# Create the output directory if it does not exist
if not os.path.isdir(PATH_OUTPUT):
    os.makedirs(PATH_OUTPUT, exist_ok=True)

##Change to directory to where you want to store the results    
os.chdir(PATH_OUTPUT)

In [14]:
os.getcwd()

'S:\\CAMELS-CH\\CAMELS-chem\\results\\catchment_aggregated_data\\isoscapes'

* #### The users should NOT change anything in the code below here. 

# Import data
* Load catchments and look at full table

*Note: data are in LV95/CH1903+, i.e. EPSG 2056*

In [15]:
catchments = gpd.read_file(catchments_shp)
catchments["bafu_id"] = catchments["gauge_id"]
catchments

,gauge_id,sensor_id,nawaf_id,nawat_id,isot_id,chirp_id,gauge_name,water_body,gauge_east,gauge_nort,gauge_lon,gauge_lat,area,area_swiss,geometry,bafu_id
0,2009,2009.0,1837.0,1837.0,NIO04,NaN,Porte du Scex,Rhône,557660,133280,6.89,46.35,5239.4,99.994914,"POLYGON Z ((2674253.038 1167429.881 0, 2674340...",2009
1,2011,2011.0,NaN,4070.0,None,NaN,Sion,Rhône,593770,118630,7.36,46.22,3372.4,100.000000,"POLYGON Z ((2674253.038 1167429.881 0, 2674340...",2011
2,2016,2016.0,1833.0,1833.0,NIO02,NaN,Brugg,Aare,657000,259360,8.19,47.48,11681.3,100.000000,"POLYGON Z ((2655969.68 1259695.589 0, 2655976....",2016
3,2018,2018.0,1835.0,1339.0,None,NaN,Mellingen,Reuss,662830,252580,8.27,47.42,3385.8,100.000000,"POLYGON Z ((2663723.38 1252919.068 0, 2663794....",2018
4,2019,2019.0,NaN,1852.0,NIO01,NaN,Brienzwiler,Aare,649930,177380,8.09,46.75,555.2,100.000000,"POLYGON Z ((2669196.412 1183579.51 0, 2669203....",2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,2617,2617.0,NaN,NaN,None,NaN,Müstair,Rom,830800,168700,10.45,46.63,128.6,42.552175,"POLYGON Z ((2820942.826 1171469.984 0, 2820953...",2617
111,2623,2623.0,NaN,NaN,None,NaN,Oberwald,Rhone,669900,154075,8.35,46.53,93.3,100.000000,"POLYGON Z ((2674253.038 1167429.881 0, 2674340...",2623
112,2634,2634.0,6169.0,1181.0,None,NaN,Emmen,Kleine Emme,663700,213630,8.28,47.07,478.3,100.000000,"POLYGON Z ((2653429.237 1216261.807 0, 2653439...",2634
113,2635,2635.0,NaN,NaN,None,NaN,"Einsiedeln, Gross",Grossbach,700710,218125,8.77,47.11,8.9,100.000000,"POLYGON Z ((2701144.527 1218073.633 0, 2701261...",2635


Now we get the isoscapes and extract these per catchment.

*Note: data are in LV95/CH1903+, i.e. EPSG 2056*

In [16]:
# Detect directories containing ASCII isoscape data
ascii_dirs = [d for d in os.listdir(GIS_dir) if "ASCII" in d]
print(ascii_dirs)

['GEO+N_13_ASCII', 'GEO+N_20_ASCII']


In [ ]:
# Extract yearly folders and file mapping
ascii_years = {}
for dirname in ascii_dirs:
    match = re.search(r'(\d{2})_ASCII$', dirname)
    if match:
        year = int("20" + match.group(1))
        full_path = os.path.join(GIS_dir, dirname)
        ascii_years[year] = sorted([
            os.path.join(full_path, f) for f in os.listdir(full_path)
            if f.endswith(".asc") and "mean" not in f
        ])

In [ ]:
for year, file_list in ascii_years.items():
    print(f"\n📅 Processing {year}...")
    year_dir = os.path.join(output_base, str(year))
    os.makedirs(year_dir, exist_ok=True)

    for _, catch in tqdm.tqdm(catchments.iterrows(), total=len(catchments), desc="Catchments"):
        catch_id = catch["gauge_id"]
        catch_geom = gpd.GeoDataFrame([catch], crs=catchments.crs)

        values = []
        months = []

        for file_path in file_list:
            month_match = re.search(r"_(\d{8})\.asc$", file_path)
            if not month_match:
                continue
            date_str = month_match.group(1)
            date = pd.to_datetime(date_str, format="%Y%m%d")
            months.append(date)

            stats = zonal_stats(catch_geom, file_path, stats="mean", nodata=-9999)
            values.append(stats[0]["mean"] if stats and stats[0]["mean"] is not None else pd.NA)

        df = pd.DataFrame({"date": months, "mean_isoscape": values})
        df.to_csv(os.path.join(year_dir, f"{catch_id}_isoscape.csv"), index=False, sep=";")

Organize the data as time-series for the dataset (stored at the results folder)

In [ ]:
folder_2013 = "../results/isoscapes/2013"
folder_2020 = "../results/isoscapes/2020"
output_folder = "../results/Dataset/catchment_aggregated_data/rain_water_isotopes"

os.makedirs(output_folder, exist_ok=True)

# List all files in 2013 (assuming they exist in 2020 too)
for filename in os.listdir(folder_2013):
    if filename.endswith(".csv"):
        path_2013 = os.path.join(folder_2013, filename)
        path_2020 = os.path.join(folder_2020, filename)
        
        if os.path.exists(path_2020):
            df_2013 = pd.read_csv(path_2013, sep=";")
            df_2020 = pd.read_csv(path_2020, sep=";")
            
            # Concatenate the two DataFrames
            merged_df = pd.concat([df_2013, df_2020], ignore_index=True)
            merged_df.columns = ["date", "delta_18o"]
            
            ## Save to output
            output_path = os.path.join(output_folder, f"camels_ch_chem_rainisotopes_{filename[0:4]}.csv")
            merged_df.to_csv(output_path, index=False)
        else:
            print(f"❗ File not found in 2020: {filename}")